In [1]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.update()
# constraints = model.getConstrs()

# constraints[0].CBasis = - 1
# constraints[1].CBasis = - 1

# # x_j.VBasis = [0, 0]
# variables = model.getVars()
# variables[0].VBasis = 0
# variables[1].VBasis = 0


# model.optimize()

Set parameter TSPort to value 61954
Set parameter TokenServer to value "10.0.1.1"
Set parameter Method to value 0


# Using .sol

In [2]:
import gurobipy as gp
import numpy as np

model = gp.Model()
model.setParam('Method', 0)

x_j = model.addMVar(2, lb=0)

# Objective
model.setObjective(x_j[0] + x_j[1], gp.GRB.MAXIMIZE)

# Constraints
model.addConstr(2 * x_j[0] + x_j[1] <= 2, "c0")
model.addConstr(x_j[0] + 2 * x_j[1] <= 2, "c1")

model.optimize()


model.write('test.sol')

# Save model
model.write('test.mps')
model.write('test.bas')

print(model.x)

Set parameter Method to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xf17fe891
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   2.000000e+00      0s
       2    1.3333333e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.333333333e+00
[0.6666666666666666, 0.6666666666666667]


In [3]:
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
Method  0
TSPort  61954

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3333333e+00   2.500000e-01   7.499993e+05      0s
       1    1.1111111e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.111111111e+00


In [4]:
model = gp.read('test.mps')
x_j = model.getVars()
model.addConstr(x_j[0] + 5* x_j[1] <= 2, "c3")
model.update()
model.read('test.bas')
# model.optimize()
# print(model.x)

Read MPS format model from file test.mps
Reading time = 0.00 seconds
: 2 rows, 2 columns, 4 nonzeros
Read basis from file test.bas


In [5]:
print(model.getVars()[0].VBasis)
print(model.getConstrs()[2].CBasis)

0
0


# Master pb

In [1]:
import gurobipy as gp
import numpy as np

path = 'GMM_quad_new/'

master_pb = gp.read(path + "output/master_pb.mps")
master_pb.read(path + 'output/master_pb.bas')
master_pb.setParam('LPWarmStart',2)
master_pb.optimize()
solution_master_pb = np.array(master_pb.x)

# Initialize slack counter
slack_counter = {constr.ConstrName: 1 if constr.Slack > 0 else 0 for constr in master_pb.getConstrs()}

Set parameter TSPort to value 61954
Set parameter TokenServer to value "10.0.1.1"
Read MPS format model from file GMM_quad_new/output/master_pb.mps
Reading time = 1.13 seconds
GMM_pb: 12753 rows, 13248 columns, 6362203 nonzeros
Read basis from file GMM_quad_new/output/master_pb.bas
Set parameter LPWarmStart to value 2
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
LPWarmStart  2
TSPort  61954

Optimize a model with 12753 rows, 13248 columns and 6362203 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+09, 1e+09]
  RHS range        [4e-03, 1e+02]
LP warm-start: use basis
Presolve time: 0.94s
Presolved: 12750 rows, 12756 columns, 89200 nonzeros

LP warm-start: get start vectors from 

In [8]:
num_characteristics = 5
num_objects = 493

In [ ]:
all_vars = master_pb.getVars()
lambda_k, u_si, p_j = all_vars[:num_characteristics], all_vars[num_characteristics: -num_objects] , all_vars[-num_objects:]


In [10]:
type(master_pb.getAttr('X',lambda_k))

list

In [4]:
type(master_pb.x)

list

In [14]:
def func(model):
    print(model.getVars()[0].x )
    print(model.getConstrs()[2].CBasis)
func(master_pb)

171.1333864927292
0


In [2]:
basic = [var for var in master_pb.getVars() if var.VBasis == 0]
non_basic = [var for var in master_pb.getVars() if var.VBasis == -1]
print(len(basic), len(non_basic))

slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == 0]
non_slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == -1]
print(len(slack), len(non_slack))

8636 4612
124877 8636


TypeError: 'dict_values' object is not subscriptable

In [3]:
master_pb.optimize()

# basic = [var for var in master_pb.getVars() if var.VBasis == 0]
# non_basic = [var for var in master_pb.getVars() if var.VBasis == -1]
# print(len(basic), len(non_basic))

# slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == 0]
# non_slack = [constr for constr in master_pb.getConstrs() if constr.CBasis == -1]
# print(len(slack), len(non_slack))


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8268 CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 48 logical processors, using up to 32 threads

Non-default parameters:
TSPort  61954

Optimize a model with 133513 rows, 13248 columns and 8299569 nonzeros
Coefficient statistics:
  Matrix range     [1e-08, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [1e+09, 1e+09]
  RHS range        [3e-04, 2e+02]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0917949e+03   0.000000e+00   0.000000e+00      1s

Solved in 0 iterations and 1.09 seconds (0.42 work units)
Optimal objective -1.091794857e+03


In [4]:
solution_master_pb = master_pb.x

Dual solution

# Pricing

In [5]:
import numpy as np

quadratic_j_j_k = np.load(path + 'data/quadratic_characteristic_j_j_k.npy')
weight_j = np.load(path + 'data/weight_j.npy')
modular_i_j_k = np.load(path + 'data/modular_characteristics_i_j_k.npy')
capacity_i = np.load(path + 'data/capacity_i.npy')
epsilon_si_j = np.load(path + 'data/epsilon_si_j.npy')

In [15]:
from pricing_notebook import pricing

In [ ]:
for i in range(90,255):
    print(i)
    modular = np.concatenate([epsilon_si_j[i][:,None]  ,modular_i_j_k[i]], axis = 1)
    results_pricing = pricing(modular, quadratic_j_j_k, weight_j, capacity_i[i], solution_master_pb, OutputFlag = False)

90
91
92
93
94
95
96
97
98
99

Interrupt request received
100



Interrupt request received
101


In [25]:
results_pricing[7:].sum()

np.float64(38.0)